In [1]:
import numpy as np
import pandas as pd
import os
import re
from collections import defaultdict

In [2]:
list_a = ['dataset', 'horizon', 'feature']
list_b = ['in', 'sd', 'fusion', 'embed', 'ff']


map = {
    'un': 'in',
    'ud': 'sd',
    'fu': 'fusion',
    'eb': 'embed',
    'ff': 'ff'
}

def parse_experiments(folder_path, top_k=None, component_names=list_a, prefix_map=map):
    rows = []

    for filename in os.listdir(folder_path):
        if not filename.endswith('.txt'):
            continue

        exp_name = filename.replace('.txt', '')
        components = exp_name.split('_')

        if len(components) != len(component_names):
            print(f"Skipping file with unexpected name: {filename}")
            continue

        component_data = dict(zip(component_names, components))

        filepath = os.path.join(folder_path, filename)
        with open(filepath, 'r') as f:
            lines = f.readlines()[1:]  # skip header

            if top_k is not None:
                lines = lines[:top_k]

            for line in lines:
                parts = line.strip().split('|')
                if len(parts) < 2:
                    continue

                config_str = parts[0].strip()
                rank_match = re.search(r'Avg Rank:\s*([0-9.]+)', parts[1])
                if not rank_match:
                    continue

                try:
                    rank = float(rank_match.group(1))
                except ValueError:
                    continue

                config_items = config_str.split('_')
                config = {}
                for item in config_items:
                    for prefix, key in prefix_map.items():
                        if item.startswith(prefix):
                            config[key] = item[len(prefix):]
                            break

                row = {**component_data, **config, 'rank': rank}
                rows.append(row)

    df = pd.DataFrame(rows)
    return df


In [3]:
# df = parse_experiments("./rank_book")
df = parse_experiments("./")

Skipping file with unexpected name: best_config_by_property_s.txt
Skipping file with unexpected name: best_config_by_property_m.txt


In [4]:
df

,dataset,horizon,feature,in,sd,fusion,embed,ff,rank
0,env,48,s,True,True,temporal,patch,rnn,2.0
1,env,48,s,True,True,feature,token,rnn,2.5
2,env,48,s,True,False,feature,invert,rnn,3.5
3,env,48,s,True,True,feature,invert,rnn,3.5
4,env,48,s,True,True,feature,none,rnn,6.0
...,...,...,...,...,...,...,...,...,...
391,social,12,s,False,False,temporal,patch,trans,92.5
392,social,12,s,False,False,feature,freq,mlp,92.5
393,social,12,s,False,True,temporal,patch,trans,94.0
394,social,12,s,False,True,feature,none,mlp,95.0


In [5]:
target_configs = [
    (True, False, 'feature', 'invert', 'trans'),
    (True, True, 'temporal', 'freq', 'mlp'),
    (True, False, 'temporal', 'patch', 'trans'),
    (False, True, 'temporal', 'none', 'mlp'),
    (False, False, 'temporal', 'token', 'trans'),
    # (False, True, 'feature', 'patch', 'trans'),
    # add more combinations as needed
]

In [6]:
df['in'] = df['in'].astype(bool)
df['sd'] = df['sd'].astype(bool)
df['fusion'] = df['fusion'].astype(str)
df['embed'] = df['embed'].astype(str)
df['ff'] = df['ff'].astype(str)

In [7]:
def is_best_config_in_list(group):
    min_rank = group['rank'].min()
    best_rows = group[group['rank'] == min_rank]
    
    for _, row in best_rows.iterrows():
        config = (
            row['in'], 
            row['sd'], 
            str(row['fusion']).strip(), 
            str(row['embed']).strip(), 
            str(row['ff']).strip()
        )
        if config in target_configs:
            return True
    return False

In [8]:
result = (
    df.groupby(['dataset', 'horizon', 'feature'])
      .apply(is_best_config_in_list)
      .reset_index(name='has_best_config')
)

print(result)

  dataset horizon feature  has_best_config
0     env      48       s            False
1   etth1      24       m            False
2   etth1      48       m            False
3  social      12       s            False


/tmp/ipykernel_2394688/2990248246.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(is_best_config_in_list)


In [9]:
count_true = result['has_best_config'].sum()
print(count_true)

0


In [10]:
def compute_rank_lag(group):
    # Get the actual best rank in the group
    best_rank = group['rank'].min()

    # Filter rows that match any of the target configs
    group['config_tuple'] = list(zip(
        group['in'].astype(bool),
        group['sd'].astype(bool),
        group['fusion'].astype(str).str.strip(),
        group['embed'].astype(str).str.strip(),
        group['ff'].astype(str).str.strip()
    ))

    matched = group[group['config_tuple'].isin(target_configs)]

    if not matched.empty:
        # If at least one config is present in the group
        lag = max(0.0, matched['rank'].min() - best_rank)
    else:
        # No matching config found in this group
        lag = None  # or np.nan if you prefer
    return pd.Series({
        'best_rank': best_rank,
        'best_config_rank': matched['rank'].min() if not matched.empty else None,
        'rank_lag': lag
    })

# Apply to each group
lag_result = df.groupby(['dataset', 'horizon', 'feature']).apply(compute_rank_lag).reset_index()

/tmp/ipykernel_2394688/2003650080.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lag_result = df.groupby(['dataset', 'horizon', 'feature']).apply(compute_rank_lag).reset_index()


In [11]:
lag_result['rank_lag'].mean()

np.float64(13.125)

In [12]:
import os
import re
import pandas as pd

list_a = ['dataset', 'horizon', 'feature']
list_b = ['in', 'sd', 'fusion', 'embed', 'ff']

prefix_map = {
    'un': 'in',
    'ud': 'sd',
    'fu': 'fusion',
    'eb': 'embed',
    'ff': 'ff'
}

# List of metric names to extract
target_metrics = ['Avg Rank', 'MSE', 'MAE', 'SMAPE', 'MASE', 'OWA']

# Build a regex that extracts the main float after metric name (ignoring rank and ±)
metric_pattern = re.compile(
    r'(' + '|'.join(re.escape(m) for m in target_metrics) + r'):\s*([0-9.]+)'
)

def parse_experiments(folder_path, top_k=None, component_names=list_a, prefix_map=prefix_map):
    rows = []

    for filename in os.listdir(folder_path):
        if not filename.endswith('.txt'):
            continue

        exp_name = filename.replace('.txt', '')
        components = exp_name.split('_')

        if len(components) != len(component_names):
            print(f"Skipping file with unexpected name: {filename}")
            continue

        component_data = dict(zip(component_names, components))

        filepath = os.path.join(folder_path, filename)
        with open(filepath, 'r') as f:
            lines = f.readlines()[1:]  # skip header

            if top_k is not None:
                lines = lines[:top_k]

            for line in lines:
                parts = line.strip().split('|')
                if len(parts) < 2:
                    continue

                config_str = parts[0].strip()
                metric_str = ' | '.join(parts[1:])  # support multiple pipes

                # Extract configuration
                config_items = config_str.split('_')
                config = {}
                for item in config_items:
                    for prefix, key in prefix_map.items():
                        if item.startswith(prefix):
                            config[key] = item[len(prefix):]
                            break

                # Extract metrics
                metrics = {}
                for match in metric_pattern.finditer(metric_str):
                    metric_name = match.group(1)
                    value = match.group(2)
                    try:
                        metrics[metric_name.lower().replace(' ', '_')] = float(value)
                    except ValueError:
                        continue

                row = {**component_data, **config, **metrics}
                rows.append(row)

    df = pd.DataFrame(rows)
    return df


In [13]:
df = parse_experiments("./rank_book")

In [14]:
df

,dataset,horizon,feature,in,sd,fusion,embed,ff,avg_rank,mse,mae,smape,mase,owa
0,PEMS04,24,s,False,True,feature,patch,trans,1.0,0.0607,0.1838,NaN,NaN,NaN
1,PEMS04,24,s,False,False,feature,invert,trans,2.0,0.0655,0.1838,NaN,NaN,NaN
2,PEMS04,24,s,False,False,feature,token,rnn,3.5,0.0687,0.1866,NaN,NaN,NaN
3,PEMS04,24,s,False,True,feature,invert,trans,6.5,0.0700,0.1916,NaN,NaN,NaN
4,PEMS04,24,s,False,True,feature,none,trans,8.5,0.0703,0.1918,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9819,exchange,336,m,False,True,temporal,token,trans,98.0,2.0236,1.1341,NaN,NaN,NaN
9820,exchange,336,m,False,True,temporal,invert,trans,99.0,2.3061,1.3069,NaN,NaN,NaN
9821,exchange,336,m,False,False,temporal,invert,trans,100.5,2.3958,1.3164,NaN,NaN,NaN
9822,exchange,336,m,False,False,temporal,none,trans,100.5,2.6140,1.3139,NaN,NaN,NaN


In [15]:
import pandas as pd
import numpy as np

def compute_metric_lag_with_percent(df, config_list, metric_cols=['avg_rank', 'mse', 'mae', 'smape', 'mase', 'owa']):
    group_cols = ['dataset', 'horizon', 'feature']
    output_rows = []

    for keys, group in df.groupby(group_cols):
        group_dict = dict(zip(group_cols, keys))

        group['config_key'] = group.apply(
            lambda row: (bool(row['in']), bool(row['sd']), str(row['fusion']), str(row['embed']), str(row['ff'])),
            axis=1
        )

        candidates = group[group['config_key'].isin(config_list)]
        if candidates.empty:
            continue

        row_result = {**group_dict}

        for metric in metric_cols:
            if metric not in group.columns:
                continue

            true_best = group[metric].min()
            list_best = candidates[metric].min()

            if pd.isna(true_best) or pd.isna(list_best):
                lag = np.nan
                pct_lag = np.nan
            else:
                lag = list_best - true_best
                # Avoid division by zero or very small values
                if abs(true_best) > 1e-12:
                    pct_lag = (lag / true_best) * 100
                else:
                    pct_lag = np.nan

            row_result[f'best_{metric}'] = true_best
            row_result[f'best_config_{metric}'] = list_best
            row_result[f'{metric}_lag'] = lag
            row_result[f'{metric}_lag_pct'] = pct_lag

        output_rows.append(row_result)

    return pd.DataFrame(output_rows)


In [16]:
lag_df = compute_metric_lag_with_percent(df, target_configs)

In [17]:
lag_df

,dataset,horizon,feature,best_avg_rank,best_config_avg_rank,avg_rank_lag,avg_rank_lag_pct,best_mse,best_config_mse,mse_lag,...,smape_lag,smape_lag_pct,best_mase,best_config_mase,mase_lag,mase_lag_pct,best_owa,best_config_owa,owa_lag,owa_lag_pct
0,M4,Daily,s,1.0,2.00,1.00,100.00,NaN,NaN,NaN,...,0.0058,0.196477,3.1000,3.1227,0.0227,0.732258,0.9575,0.9620,0.0045,0.469974
1,M4,Hourly,s,4.0,14.33,10.33,258.25,NaN,NaN,NaN,...,0.7658,4.333728,2.4683,3.3377,0.8694,35.222623,1.0115,1.1982,0.1867,18.457736
2,M4,Monthly,s,2.0,3.00,1.00,50.00,NaN,NaN,NaN,...,0.0000,0.000000,0.9193,0.9253,0.0060,0.652671,0.8682,0.8693,0.0011,0.126699
3,M4,Quarterly,s,1.0,2.00,1.00,100.00,NaN,NaN,NaN,...,0.0230,0.231983,1.1500,1.1542,0.0042,0.365217,0.8693,0.8722,0.0029,0.333602
4,M4,Weekly,s,1.0,10.00,9.00,900.00,NaN,NaN,NaN,...,0.4885,5.206224,2.7800,2.9330,0.1530,5.503597,1.0125,1.0667,0.0542,5.353086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,weather,336,s,2.5,8.00,5.50,220.00,0.0016,0.0017,0.0001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,weather,720,m,10.0,22.50,12.50,125.00,0.3321,0.3321,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,weather,720,s,1.5,1.50,0.00,0.00,0.0021,0.0021,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,weather,96,m,1.0,30.50,29.50,2950.00,0.1588,0.1654,0.0066,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
def average_percentage_lag(df, metrics=['mse', 'mae', 'smape', 'mase', 'owa']):
    pct_cols = [f'{m.lower()}_lag_pct' for m in metrics]
    # Calculate the mean % lag for each metric, skipping NaNs
    avg_pct_lag = df[pct_cols].mean(skipna=True)
    return avg_pct_lag


In [19]:
# Assuming `result_df` is from compute_metric_lag_with_percent
metrics = ['MSE', 'MAE', 'SMAPE', 'MASE', 'OWA']
avg_pct_lag = average_percentage_lag(lag_df, metrics)
print(avg_pct_lag)


mse_lag_pct      9.223177
mae_lag_pct      4.849635
smape_lag_pct    1.661402
mase_lag_pct     7.079394
owa_lag_pct      4.123516
dtype: float64


In [20]:
avg_pct_lag.mean()

np.float64(5.387424847123938)